In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os
import random
import tensorflow as tf

def seed_everything(seed:int=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

# 데이터 로드

## user 데이터

In [3]:
user_column_names =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
user_df = pd.read_csv('../data/ml-100k/u.user', sep='|', names=user_column_names)

In [4]:
user_df.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [8]:
print(user_df.shape)
print(user_df['user_id'].count())

(943, 5)
943


In [9]:
user_df = user_df.drop(columns = ['zip_code'])
user_df.head()

,user_id,age,sex,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


## item 데이터

In [10]:
item_column_names = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

item_df = pd.read_csv('../data/ml-100k/u.item', sep='|', names=item_column_names, encoding='latin-1')
item_df.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
drop_columns_now = ['release_date','video_release_date','IMDb_URL']
item_df = item_df.drop(columns = drop_columns_now)
item_df.head()

,movie_id,movie_title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


## Score 데이터

In [12]:
score_df = pd.read_csv('../data/ml-100k/u.data', sep="\t", header=None)
score_df.columns = ['user id', 'movie id', 'rating', 'timestamp']
score_df.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
print(score_df.shape)

(100000, 4)


# Data Preprocessing - Ongoing
* implicit data 만들기 : user-item 상호작용 matrix에서 interaction이 있으면 1(positive), 없으면 0(negative)
* negative, positive 데이터 분리

# 모델 구조
reference : 

https://github.com/LeeHyeJin91/Neural_CF

https://github.com/hexiangnan/neural_collaborative_filtering


In [ ]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

## GMF

In [ ]:
# GMF - by Sunny
from tensorflow.keras.layers import *
def gmf_model(user_num,item_num):
    # input
    user_input = Input(1,)
    item_input = Input(1,)

    # embedding 
    user_embedding = Embedding(user_num, 32, name = 'user_embedding')
    item_embedding = Embedding(item_num, 32, name = 'item_embedding')

    # flatten
    user_flatten = Flatten(user_embedding)
    item_flatten = Flatten(item_embedding)

    # Merge
    x = Multiply([user_flatten,item_flatten])

    # output
    pred = Dense(1, activation = 'sigmoid',kernel_initializer='lecun_uniform', name='output_layer')(x)

    model = Model(input = [user_input,item_input], output=pred)
    model.compile(optimizer='adam', loss='binary_crossentropy')

## MLP

In [ ]:
# MLP - by Sunny
def mlp_model(user_num,item_num):
    # input
    user_input = Input(1,)
    item_input = Input(1,)

    # embedding
    user_embedding = Embedding(user_num, 32, name = 'user_embedding')
    item_embedding = Embedding(item_num, 32, name = 'item_embedding')

    # flatten
    user_flatten = Flatten(user_embedding)
    item_flatten = Flatten(item_embedding)
    
    # Merge
    x = Concatenate([user_flatten, item_flatten])
    
    # Multi layers
    x = Dense(64, kernel_regularizer ='l2', activation = 'relu', name = 'layer1')(x)
    x = Dense(32, kernel_regularizer ='l2', activation = 'relu', name = 'layer2')(x)
    x = Dense(16, kernel_regularizer ='l2', activation = 'relu', name = 'layer3')(x)

    # output layer
    pred = Dense(1, activation = 'sigmoid',kernel_initializer='lecun_uniform', name='output_layer')(x)
    #lecun_uniform : [-limit, limit] 범위내 균등분포에 따라 샘플 생성

    model = Model(input = [user_input,item_input], output=pred)
    model.compile(optimizer='adam', loss='binary_crossentropy')

## NeuMF

In [ ]:
def neumf_model(user_num,item_num):
    user_input = Input(1,)
    item_input = Input(1,)

    # gmf embedding
    gmf_user_embedding = Embedding(user_num, 32, name = 'gmf_user_embedding')
    gmf_item_embedding = Embedding(item_num, 32, name = 'gmf_item_embedding')

    # gmf flatten
    gmf_user_flatten = Flatten(gmf_user_embedding)
    gmf_item_flatten = Flatten(gmf_item_embedding)

    # gmf merge
    gmf_merge = Multiply([gmf_user_flatten,gmf_item_flatten])

    # mlp embedding
    mlp_user_embedding = Embedding(user_num, 32, name = 'user_embedding')
    mlp_item_embedding = Embedding(item_num, 32, name = 'item_embedding')

    # mlp flatten
    mlp_user_flatten = Flatten(mlp_user_embedding)
    mlp_item_flatten = Flatten(mlp_item_embedding)

    # mlp merge
    mlp_merge = concatenate([mlp_user_embedding, mlp_item_embedding])

    # mlp layers
    x = Dense(64, kernel_regularizer ='l2', activation = 'relu', name = 'layer1')(mlp_merge)
    x = Dense(32, kernel_regularizer ='l2', activation = 'relu', name = 'layer2')(x)
    x = Dense(16, kernel_regularizer ='l2', activation = 'relu', name = 'layer3')(x)

    # MLP + GMF
    mlp_gmf = concatenate([x, gmf_merge])

    # output layer
    output_layer = Dense(1, activation='sigmoid', kernel_initializer = 'lecun_uniform', name=output_layer)(mlp_gmf)

    # model
    model = Model([user_input,item_input], output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy')


# 평가 지표

In [ ]:
import numpy as np
import heapq

class Metrix:
    def __init__(self):
        pass
    
    def get_hits(self, k_ranked, holdout):
        for item in k_ranked:
            if item == holdout:
                return 1
        return 0
    
    def eval_rating(self, idx, test_ratings, test_negatives, K, model):
        # holdout(df_Test의 item)이 k순위 내에 있는지 평가하는 함수
        items = test_negatives[idx]
        user_idx = test_ratings[idx][0]
        holdout = test_ratings[idx][1]
        items.append(holdout)
        
        # prediction
        predict_user = np.full(len(items), user_idx, dtype='int32').reshape(-1,1)
        np_items = np.array(items).reshape(-1,1)
        
        predictions = model.predict([predict_user, np_items])
        predictions = predictions.flatten().tolist()
        item_to_pre_score = {item:pre for item, pre in zip(items, predictions)}
        
        # 점수가 높은 상위 k개 아이템 리스트
        k_ranked = heapq.nlargest(K,item_to_pre_score, key=item_to_pre_score.get)
        
        # holdout이 상위 K순위에 포함되는지
        # {1: 포함, 0: 포함x}
        hits = self.get_hits(k_ranked, holdout)
        
        return hits
    
    def evaluate_top_k(self, df_neg, df_test, model, K=10):
        # Top- K metric을 사용해 모델 평가하는 함수
        hits = []
        test_u = df_test['user_id'].values.tolist()
        test_i = df_test['item_id'].values.tolist()
        
        test_ratings = list(zip(test_u, test_i))
        df_neg = df_neg.drop(df_neg.columns[0], axis=1)
        test_negatives = df_neg.values.tolist()
        
        # user 샘플링
        sample_idx_lst = np.random.choice(len(test_ratings), int(len(test_ratings)))
        for user_idx in sample_idx_lst:
            
            hitrate = self.eval_rating(user_idx, test_ratings, test_negatives, K, model)
            hits.append(hitrate)
            
        return hits
        